# tensorflow로 글자 단위의 언어 모델 구현

앞으로 만들 모델의 입력은 텍스트 문장입니다. 목표는 입력 문서와 비슷한 새로운 텍스트를 생성하는 모델을 개발하는 것입니다. 데이터 전처리, RNN 모델 구성, 다음 글자를 예측하고 새로운 텍스트를 생성하는 세 개의 단계로 나누어 구현하겠습니다.

## 데이터 전처리

사용할 데이터는 수천 권의 무료 전자책을 제공하는 구텐베르크 프로젝트 웹 사이트에서 구한 윌리업 셰익스피어의 <햄릿> 텍스트 입니다.

In [1]:
with open('pg2265.txt', 'r', encoding='utf-8') as f:
    text = f.read()

파이선 세션에서 준비된 텍스트를 읽습니다. 그 다음 각 글자와 정수를 매핑한 딕셔너리 char2int, 거꾸로 정수와 고유한 글자를 매핑한 int2char 딕셔너리를 만듭니다. 그리고 char2int 딕셔너리를 사용하여 텍스트를 넘파이 정수 배열로 변환합니다.

In [2]:
text = text[16247:] # 구텐브르크 프로젝트의 법률 조항 부분 제외
chars = set(text)
char2int = {ch: i for i, ch in enumerate(chars)}
int2char = dict(enumerate(chars))
text_ints = np.array([char2int[ch] for ch in text],
                     dtype=np.int32)

In [3]:
len(chars), len(text), len(text_ints)

(65, 162850, 162850)

다음으로 할 단계는 위 데이터를 시퀀스의 배치로 바꾸는 작업입니다. 이 작업이 데이터 전처리에서 가장 중요한 단계라고 할 수 있습니다. 지금까지 본 글자 시퀀스를 기반으로 새로운 글자를 예측하는 것이 목적이므로, 신경망의 입력(x)과 출력(y)을 한 글자씩 이동합니다.

In [26]:
def reshape_data(sequence, batch_size, num_steps):
    mini_batch_length = batch_size * num_steps
    num_batches = int(len(sequence) / mini_batch_length)
    if num_batches*mini_batch_length + 1 > len(sequence):
        num_batches = num_batches - 1
    # 전체 배치에 포함되지 않는 시퀀스의 끝부분은 삭제합니다
    x = sequence[0: num_batches*mini_batch_length]
    y = sequence[1: num_batches*mini_batch_length + 1]
    # x와 y를 시퀀스 배치의 리스트로 나눕니다
    x_batch_splits = np.split(x, batch_size)
    y_batch_splits = np.split(y, batch_size)
    # 합쳐진 배치의 크기는 batch_size x mini_batch_length가 됩니다
    x = np.stack(x_batch_splits)
    y = np.stack(y_batch_splits)
    
    return x, y

In [27]:
train_x, train_y = reshape_data(text_ints, 64, 10)

In [31]:
print(train_x.shape)
print(train_x[0, :10])
print(train_y[0, :10])
print(''.join(int2char[i] for i in train_x[0, :10]))
print(''.join(int2char[i] for i in train_y[0, :10]))

(64, 2540)
[48  1 22 42 48 53  6 14 22 40]
[ 1 22 42 48 53  6 14 22 40 43]
The Traged
he Tragedi


출력 결과 한 글자씩 밀려있다는 것을 확인할 수 있습니다. 다음 단계에서 x와 y를 나누어 열 길이가 스텝 횟수와 동일한 배치를 출력하는 create_batch_generator 함수를 정의합니다. 나중에 이 제너레이너를 사용하여 네트워크를 훈련하는 동안 미니 배치를 반복하겠습니다. 

In [37]:
def create_batch_generator(data_x, data_y, num_steps):
    batch_size, tot_batch_length = data_x.shape[0:2]
    num_batches = int(tot_batch_length/num_steps)
    for b in range(num_batches):
        yield (data_x[:, b*num_steps:(b+1)*num_steps],
               data_y[:, b*num_steps:(b+1)*num_steps])

이 코드에서 정의한 제너레이터는 메모리 부족을 해결할 수 있는 아주 좋은 기법입니다. 신경망을 훈련하는 동안 모든 데이터를 미리 나누어 메모리에 저장하지 않고 데이터셋을 미니 배치로 나누는 방식이 바람직합니다.

train_x와 train_y의 배열에서 길이 100까지만 사용하여 배치 데이터를 텍스트로 만들어 보겠습니다. 시퀀스 길이는 15로 설정하겠습니다. 길이가 100이므로 제너레이터 함수는 길이가 15인 시퀀스의 배치를 6번 반환할 것입니다.

In [38]:
bgen = create_batch_generator(train_x[:, :100], train_y[:, :100], 15)
for x, y in bgen:
    print(x.shape, y.shape, end='  ')
    print(''.join(int2char[i] for i in x[0, :]).replace('\n', '*'), '   ',
          ''.join(int2char[i] for i in y[0, :]).replace('\n', '*'))

(64, 15) (64, 15)  The Tragedie of     he Tragedie of 
(64, 15) (64, 15)   Hamlet**Actus      Hamlet**Actus P
(64, 15) (64, 15)  Primus. Scoena      rimus. Scoena P
(64, 15) (64, 15)  Prima.**Enter B     rima.**Enter Ba
(64, 15) (64, 15)  arnardo and Fra     rnardo and Fran
(64, 15) (64, 15)  ncisco two Cent     cisco two Centi


reshape 메서드를 사용하여 text_ints 배열을 64개의 배치 행을 가진 형태로 바꿉니다.

In [39]:
batch_size = 64
num_steps = 100
train_x, train_y = reshape_data(text_ints, batch_size, num_steps)
print(train_x.shape, train_y.shape)

(64, 2500) (64, 2500)


데이터 전처리의 마지막 단계는 이 데이터를 one hot encoding으로 바꾸는 작업입니다.  
  
이전 감성분석의 예제에서는 embedding 층을 사용하여 단어를 길이가 200인 벡터로 인코딩했습니다. 이 때 타깃 데이터는 긍정 또는 부정 리뷰를 나타내는 1차원 배열입니다. 일련의 시퀀스를 처리한 후 손실 함수로부터 그래디언트를 계산했습니다.  
  
글자 단위의 RNN 모델에서는 조금 다른 방식을 사용합니다. 모델에서 처리하는 글자마다 그래디언트를 모두 계산하여 사용하겠습니다. 이렇게 하려면 타깃 데이터도 전체 타입 스텝에 걸쳐 one hot encoding 되어야 합니다. tensorflow에서 제공하는 `to_categorical` 함수를 사용해보도록 하겠습니다.

In [41]:
from tensorflow.keras.utils import to_categorical

In [42]:
train_encoded_x = to_categorical(train_x)
train_encoded_y = to_categorical(train_y)
print(train_encoded_x.shape, train_encoded_y.shape)

(64, 2500, 65) (64, 2500, 65)


정수값이 0부터 시작한다고 가정하므로 one_hot encoding 벡터의 길이는 최댓값에 1을 더해야 합니다.

In [44]:
print(np.max(train_x), np.max(train_y))

64 64


## 글자 단위 RNN 모델 만들기

In [46]:
from tensorflow.keras import models, layers

In [47]:
char_model = models.Sequential()

이번에는 훈련 데이터를 one hot encoding 했으므로 embedding 층 대신 LSTM 층을 바로 추가합니다. 이 때 다음의 두 가지 사항에 대해 고려해야합니다.

첫째, 이 모델은 훈련할 때 길이가 100인 시퀀스를 주입합니다. 즉, 타입 스텝의 길이가 100입니다. 하지만 새로운 글자를 생성할 때는 이전 글자를 주입하여 한 글자씩 생성합니다. 다시 말해 샘플링 시에는 배치 크기가 1이 됩니다. 정리하자면 샘플링 모드일 때 batch_size=1, num_step=1 그리고 훈련 모드일 때 batch_size=64, num_steps=100 입니다. 흔히 이런 RNN 네트워크의 구조를 "시간에 따라 동적으로 펼친다"라고 말합니다. 일반적으로 모델에 추가하는 첫 번쨰 층에는 input_shape 매개변수로 배치 차원을 제외한 입력 크기를 지정해야 합니다. LSTM 층에서 가변 길이 시퀀스를 처리하려면 타입 스텝 길이에 해당하는 input_shape의 첫 번째 차원을 None으로 지정하면 됩니다. 두 번째 차원은 one hot encoding 벡터의 크기가 됩니다.  
  
둘째, 모든 타임 스텝에 대해 그래디언트를 계산하여 모델을 업데이트할 것입니다. 따라서 LSTM 층이 시퀀스의 마지막 타입 스텝의 출력만 반환하지 않고 전체 시퀀스에 대해 출력을 만들어야 합니다. 이렇게 하려면 앞서 언급한 대로 LSTM 층의 `return_sequences` 매개변수를 True로 지정해야 합니다.  
  
이런 점을 고려하여 다음과 같이 128개의 순환 유닛을 가진 LSTM 층을 모델에 추가합니다.

In [48]:
num_classes = len(chars) # 이 모델에 입력할 데이터는 num_classes 크기로 one hot encoding 되었음
char_model.add(layers.LSTM(128, input_shape=(None, num_classes), 
                           return_sequences=True))

지금까지는 Dense 층을 추가하기 전에 Flatten 층을 추가했습니다. Dense 층은 전형적으로 2차원 텐서를 다루는데 활용됩니다. 하지만 이번에는 모든 타임 스텝에 대한 손실을 계산해야 하기 때문에 LSTM 층에서 출력되는 3차원 텐서를 그대로 다루어야 합니다. LSTM 층에서 출력되는 텐서 크기는 (배치 개수, 타임 스텝 개수, 순환 유닛 개수)입니다. Dense 층을 통과할 때 이 텐서의 첫 번째와 두 번째 차원이 유지되어야 합니다.

이 작업을 처리하기 위해 Flatten 층을 추가하지 않고 LSTM 층의 출력을 타임 스텝 순으로 Dense 층에 주입하고, 결과를 받아 다시 타임 스텝 순서대로 쌓아야 합니다. `tf.keras.layers.TimeDistributed` 클래스를 사용하면 Dense 층을 감싸서 타임 스텝을 가진 입력을 다룰 수 있습니다.

In [49]:
char_model.add(layers.TimeDistributed(layers.Dense(num_classes,
                                                   activation='softmax')))

In [50]:
char_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 128)         99328     
_________________________________________________________________
time_distributed (TimeDistri (None, None, 65)          8385      
Total params: 107,713
Trainable params: 107,713
Non-trainable params: 0
_________________________________________________________________


가변 길이 시퀀스를 다루기 위해 LSTM 층과 TImeDistributed 층의 출력에서 두 번째 차원이 None으로 된 것을 볼 수 있습니다.

## 글자 단위 RNN 모델 훈련

이번에는 그래디언트 폭주를 피하기 위한 대표적인 방법인 그래디언트 클리핑을 적용해 보겠습니다. 옵티마이저 클래스의 객체를 직접 만들어 모델의 compile 메서드에 전달하는 방식으로 진행합니다.

In [51]:
from tensorflow.keras.optimizers import Adam

In [53]:
adam = Adam(clipnorm=5.0)

`tf.keras.optimizers`에 있는 옵티마이저들은 그래디언트 클리핑을 위한 두 개의 매개변수를 제공합니다. 하나는 L2 nrom의 임계값을 지정하는 `clipnorm`이고 다른 하나는 절댓값으로 임계값을 지정하는 `clipvalue`입니다. 두 방식을 동시에 사용하는 것도 가능합니다.

이번에는 65개의 글자에 대한 확률을 출력하는 다중 클래스 모델이므로 손실함수는 `categorical_crossentropy`를 사용합니다.

In [54]:
char_model.compile(loss='categorical_crossentropy', optimizer=adam)

훈련한 모델을 저장하여 나중에 학습을 이어가거나 텍스트를 생성할 수 있도록 체크포인트 콜백을 준비합니다.

In [55]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

In [56]:
callback_list = [ModelCheckpoint(filepath='char_rnn_checkpoint.h5')]

이제 500번의 에포크 동안 모델을 훈련하겠습니다. Sequential 모델은 입력과 타겟 배치를 반환하는 제너레이터와 함께 쓸 수 있는 `fit_generator` 메서드를 제공합니다. 앞서 만든 create_batch_generator 함수로부터 제너레이터 객체를 만들어 `fit_generator` 메서드에 전달합니다.  
  
단, `fit_generator` 메서드는 파이썬 제너레이터에서 배치를 끝없이 반환할 것으로 기대합니다. 데이터가 끝없이 생성되므로 하나의 에포크를 정의하기 위해 제너레이터로부터 몇 번이나 배치를 뽑을 것인지 알려주어야 합니다. `fit_generator` 메서드의 `steps_per_epoch` 매개변수에서 이를 설정합니다.

In [58]:
for i in range(500):
    bgen = create_batch_generator(train_encoded_x,
                                  train_encoded_y, num_steps)
    char_model.fit_generator(bgen, steps_per_epoch=25, epochs=1,
                             callbacks=callback_list, verbose=0)

이 예제에서는 시퀀스 길이가 100이므로 전부 25번의 배치가 생성됩니다. create_batch_generator 함수는 배치를 순환하지 않기 때문에 25번째 배치 이후에는 더 이상 추출하지 못하는 에러가 발생합니다. 이를 해결하기 위해 for 반복문에서 fit_generator 메서드를 호출할 때 epochs를 1로 설정합니다. 전체 훈련 횟수는 500번이고 훈련할 때마다 제너레이터를 다시 초기화해야 합니다.

---